## IPFS Client
This notebook interacts with our local IPFS node (and, consequently, the configured IPFS private network(s)), using our Python module, as to simulate a Jupyter Server extension (i.e. the backend).

### Python module setup

In [ ]:
import jcipfsclient as ipfs

### Verify local communication to our IPFS node

In [18]:
import os
node = os.getenv('IPFS_NODE')
nodeApiUrl = 'http://' + node + ':5001'
ipfs.getId(nodeApiUrl)

'12D3KooWRKf8SgXjpRBwDbaYwYEHudufhcYc26X43Snr6hdxSxWF'

### Join the IPFS network
Using the configuration [obtained via Fabric](./Fabric.ipynb#Configure-IPFS), we will join our local IPFS node to the private IPFS network.

In [22]:
ipfs.joinNetwork(
  nodeApiUrl,
  '/mnt/ipfs',
  '/key/swarm/psk/1.0.0/\n/base16/\n63e8d44cb8d738ece5681d42dc918ff882cbe28458d81f64e764f95e3f77929f',
  [
      '/dns4/peer0.pnet0.orga.ipfs.localhost/tcp/4001/p2p/12D3KooWCLVuku1xQZLfhwn4UkkR9mFWw1bDeVJ2XVpwzEpvC5JV',
      '/dns4/peer0.pnet0.orgb.ipfs.localhost/tcp/4001/p2p/12D3KooWFJ4XnjSowo6nCTiMPKaNLXrsDakkM5RQiPpZdDSp4uZM',
      '/dns4/peer0.pnet0.orgc.ipfs.localhost/tcp/4001/p2p/12D3KooWNB1Sww8vDHxhg8rtiNc1Ra26ECT3nS7URVbLUkhpZ6fB'
  ]
)

**Optional: add a relay**  
If we are using an external JupyterLab instance (and its IPFS node client), we need to configure access to an IPFS relay (since our local IPFS node does not have direct access to our private IPFS network). Using the configuration obtained via Fabric, we will set up our local IPFS node to use a relay to reach the private IPFS network.

In [5]:
ipfs.configRelay(
  nodeApiUrl,
  [
    '/dns4/relay0.pnet0.orga.ipfs.localhost/tcp/4002/p2p/12D3KooWRsg23DzqhXgt7yfzVf2fs3PPHrRmt6e49p94CgTNcXmh'
  ]
)

Verify if our IPFS node can detect any peers on the network:

In [7]:
ipfs.getPeers(nodeApiUrl)

[{'peer': '12D3KooWMvugsCa9vFRrQFv2fcTa3ZbKxHHfRrZnwwdquWV6HwBE',
  'addr': '/ip4/172.20.3.2/tcp/4001'},
 {'peer': '12D3KooWFJ4XnjSowo6nCTiMPKaNLXrsDakkM5RQiPpZdDSp4uZM',
  'addr': '/ip4/172.20.3.4/tcp/4001'},
 {'peer': '12D3KooWRsg23DzqhXgt7yfzVf2fs3PPHrRmt6e49p94CgTNcXmh',
  'addr': '/ip4/172.20.3.45/tcp/4002'},
 {'peer': '12D3KooWPyNaEskA1tyz1DRGubFf8BeKGuHNRbzJkxjb1GKWmZNt',
  'addr': '/ip4/172.20.3.45/tcp/4002/p2p/12D3KooWRsg23DzqhXgt7yfzVf2fs3PPHrRmt6e49p94CgTNcXmh/p2p-circuit'},
 {'peer': '12D3KooWLNH1D5z5VEt1ftHZe7M7j8aeGZAGTT1aNrBEDh2MswBZ',
  'addr': '/ip4/172.20.3.47/tcp/4001'},
 {'peer': '12D3KooWNB1Sww8vDHxhg8rtiNc1Ra26ECT3nS7URVbLUkhpZ6fB',
  'addr': '/ip4/172.20.3.6/tcp/4001'},
 {'peer': '12D3KooWDwgActjXPDopdz4wmUGegY63KQjEdHJ3F7WpJK7KdDxG',
  'addr': '/ip4/172.20.3.7/tcp/4001'},
 {'peer': '12D3KooWCLVuku1xQZLfhwn4UkkR9mFWw1bDeVJ2XVpwzEpvC5JV',
  'addr': '/ip4/172.20.3.8/tcp/4001'},
 {'peer': '12D3KooWLLw213PNdKv5U7KpxMHvYhLnFca6HMc9QxFbBDTEFViP',
  'addr': '/ip4/172.20

### Add a file
Now that we have configured our IPFS network access, we can add a file from our local system (i.e. the JupyterLab backend storage) to IPFS. Once we have added a file to IPFS, we can [publish](./Fabric.ipynb#Add-a-data-description) its metadata on Fabric (note that the file can be automatically encrypted using a random self-generated generated key).

In [14]:
!echo 'This is a test file from org C.' > testorgc.txt

In [15]:
ipfs.addFile(nodeApiUrl, './testorgc.txt')

{'cid': 'QmRrGntXMyZoVcBgA2QAwn6rRn8vsXxLDVuqFma44URMu2',
 'base64Key': 'nR9WmUk+Wzkbb8d1+IEkTAr6nGjruVJmXRgk4N/6BSw=',
 'cipherMode': 'ChaCha20'}

### Retrieve a file
We can now switch to another JupyterLab instance and, after [repeating](./Fabric.ipynb) the steps to create another Fabric identity and configuring our local IPFS node, [retrieve](./Fabric.ipynb#Retrieve-a-data-description) the file's metadata (CID and encryption info) from Fabric. Then we can use IPFS to download our file.

In [16]:
filename = 'testorgc.txt'
cid = 'QmRrGntXMyZoVcBgA2QAwn6rRn8vsXxLDVuqFma44URMu2'
cryptKey = 'nR9WmUk+Wzkbb8d1+IEkTAr6nGjruVJmXRgk4N/6BSw='
chunkSize = 10485760
cipher = 'ChaCha20'
ipfs.getFile(nodeApiUrl, cid, filename, cryptKey, chunkSize, cipher)

In [17]:
!cat testorgc.txt

This is a test file from org C.


### Pinning Service
In the example above our added file is *only* available if our IPFS node is connected to the specified IPFS private network. However, we can use our organization's [provided](./Fabric.ipynb#Configure-IPFS) IPFS Pinning Service to store our (encrypted) file on our organization's IPFS cluster. This guarantees there's always an IPFS peer (or peers) available to provide our file.

In [18]:
cid = 'QmRrGntXMyZoVcBgA2QAwn6rRn8vsXxLDVuqFma44URMu2'
pinningServiceUrl = 'https://cluster0.pnet0.orgc.ipfs.localhost:9097'
pinningServiceUser = 'orgc'
pinningServicePass = '7342764f480A9feb8abd0F51463160'
pinningServiceCert = '/mnt/crypto-config/ipfs/orgc/pnet0/cluster0/pinsvcapi.crt'
requestId = ipfs.addRemotePin(pinningServiceUrl, cid, pinningServiceUser, pinningServicePass, pinningServiceCert)
print(requestId)

QmRrGntXMyZoVcBgA2QAwn6rRn8vsXxLDVuqFma44URMu2


In [23]:
ipfs.getRemotePinStatus(pinningServiceUrl, requestId, pinningServiceUser, pinningServicePass, pinningServiceCert)

'pinned'

### Storage maintenance
Our local IPFS node contains a copy of all encrypted files we have downloaded. We can manually clean up these files immediately to claim back storage space.

In [ ]:
ipfs.collectGarbage(nodeApiUrl)

**Removing local pins**  
By default, all our added (encrypted) files are always preserved by our local IPFS node (even after a garbage collection). We can manually remove these pins from our node. However, as long as the file's metadata (i.e. the encryption key) [is still available on Fabric](./Fabric.ipynb#Delete-a-data-description), and some other IPFS peer has a copy of the file, the file can still be accessed.

In [ ]:
ipfs.getPins(nodeApiUrl)

In [ ]:
ipfs.rmPin(nodeApiUrl, cid)